In [6]:
!pip install sightengine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import json
import os
import timeit
from sightengine.client import SightengineClient


In [14]:
# -*- coding: utf-8 -*-

'''
 1. Load the input file
'''
vidfile=str(input("Enter the video file to be moderated: "))
(name,ext)=os.path.splitext(vidfile)


base_url="https://github.com/NikhilBhargav/sightEngine/blob/master/input/"
#base_url="https://github.com/cygnetmllab01/sightEngine/tree/master/input/"
url=base_url+vidfile+"?raw=true"
#print(url)
    
'''
 2. Call SightEngine's Video moderation API on input video for all models
''' 
client = SightengineClient('1406150133', 'rjMYsuTWX3Ksy8XXqar3')

#Time of execution of Videomoderation API
iterTime=0.0
starttime = timeit.default_timer()

#Checking only Nudity; weapons, alcohol and drugs; offensive and minors
output = client.check('nudity','wad','face-attributes','offensive').video_sync(url)
endtime=timeit.default_timer()
iterTime= endtime - starttime

json_object=json.dumps(output, indent = 2)
print(json_object)   

'''
 3. Store the output as json output file in separate output folder
''' 
cwd = os.getcwd()
folder_name="output"

vidpath=os.path.join(cwd,folder_name)

# create a directory to store the audio chunks
if not os.path.isdir(folder_name):
    os.mkdir(folder_name)

file=name+'.json'
out_file = os.path.join(vidpath, file)   
with open(out_file, 'w', encoding='utf-8') as f:
    json.dump(output, f, ensure_ascii=False, indent=2)
    
'''
 4. Use timeit to calculate moderation time
'''
#print("Starttime:",starttime,"Endtime:",endtime,"Exec Time:",endtime-starttime)
print("Time taken to moderate", vidfile,"is:",iterTime,"usec")

#Set Flags as True unless proved
nudityFlag= False
wepAlcoDrugFlag= False
offensiveFlag= False

thresholdProbability=0.75

#Process only if you get success of moderationAPI
if (output["status"]=="success"):
    data_dict=output["data"]
    mod_list=data_dict["frames"]
    for entry in mod_list:
        #1. Check for WepAlcoDrugFlag
        if(entry["weapon"]>=thresholdProbability or entry["alcohol"]>=thresholdProbability or entry["drugs"]>=thresholdProbability):
            wepAlcoDrugFlag=True
            
        #2. Check for NudityFlag
        if( (entry["nudity"]["raw"]>=thresholdProbability) or ((entry["nudity"]["partial"]>=thresholdProbability))): 
            nudityFlag=True
            
        #3. Check for OffensiveFlag
        if(entry["offensive"]["prob"]>=thresholdProbability):
            offensiveFlag=True
    
if(nudityFlag==False and wepAlcoDrugFlag==False and offensiveFlag==False):
    print("The given video is Approved")
else:
    reason=""
    if (nudityFlag):
        reason="Nudity"
    if (wepAlcoDrugFlag):
        if(len(reason)!=0):
            reason=reason+", Weapon, Alcohol and Drugs"
        else:
            reason="Weapon, Alcohol and Drugs"
    if (offensiveFlag):
        if(len(reason)!=0):
            reason=reason+" and Offensive content."
        else:
            reason="Offensive content."
        
    print("The given video is Rejected due to",reason)

Enter the video file to be moderated: VID-20200902-WA0001.mp4
{
  "status": "success",
  "request": {
    "id": "req_cQjJTF3jiRzapBL7dSJQS",
    "timestamp": 1666891625.123073,
    "operations": 28
  },
  "media": {
    "id": "med_cQjJ5MkpoxzyY2yNjc797",
    "uri": "https://github.com/NikhilBhargav/sightEngine/blob/master/input/VID-20200902-WA0001.mp4?raw=true"
  },
  "data": {
    "frames": [
      {
        "info": {
          "id": "med_cQjJ5MkpoxzyY2yNjc797_1",
          "position": 0
        },
        "nudity": {
          "raw": 0.01,
          "safe": 0.98,
          "partial": 0.01
        },
        "weapon": 0.01,
        "alcohol": 0.01,
        "drugs": 0.01,
        "faces": [],
        "offensive": {
          "prob": 0.01
        }
      },
      {
        "info": {
          "id": "med_cQjJ5MkpoxzyY2yNjc797_2",
          "position": 2000
        },
        "nudity": {
          "raw": 0.01,
          "safe": 0.98,
          "partial": 0.01
        },
        "weapon": 